In [10]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

In [11]:
from datasets import load_dataset
from seamless_communication.inference import Translator
from jiwer import wer
from sacrebleu import corpus_bleu

In [12]:
# Initialize a Translator object with a multitask model, vocoder on the GPU.

model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda"), # Changed from "cuda:0" to "cpu"
    dtype=torch.float16,
)

Using the cached checkpoint of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached checkpoint of vocoder_v2. Set `force` to `True` to download again.
/scratch/aj/Bhavna/bhav_venv_311/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [13]:
from datasets import load_dataset
import torch
import torchaudio

references_all = {}
hypotheses_s2tt_all = {}
hypotheses_t2tt_all = {}
predicted_text_from_s2s_all = {}
predicted_text_from_t2s_all = {}

# Language mapping: SeamlessM4T tgt_lang → (FLEURS target_lang)
lang_map = {
    "tel": "te_in",   # Telugu
    "urd": "ur_pk"    # Urdu (Pakistan variant)
}

# Fixed source language
fleurs_src_lang = "hi_in"  # Hindi in FLEURS
sm4t_src_lang = "hin"      # Hindi in SeamlessM4T

def resample_to_16k(audio, orig_sr):
    # implement resampling logic here, e.g. torchaudio.transforms.Resample
    return torchaudio.transforms.Resample(orig_sr, 16000)(torch.tensor(audio)).numpy()

for sm4t_tgt_lang, fleurs_tgt_lang in lang_map.items():
    print("\n" + "="*50)
    print(f"🔹 Processing Target Language: {sm4t_tgt_lang.upper()} ({fleurs_tgt_lang})")
    print("="*50)

    # Load FLEURS test splits
    src_dataset = load_dataset("google/fleurs", fleurs_src_lang, split="test")
    tgt_dataset = load_dataset("google/fleurs", fleurs_tgt_lang, split="test")

    # Build mappings from sentence_id (the 'id' field) to record
    src_by_id = {item["id"]: item for item in src_dataset}
    tgt_by_id = {item["id"]: item for item in tgt_dataset}

    # Compute intersection of available sentence_ids
    common_ids = sorted(set(src_by_id.keys()) & set(tgt_by_id.keys()))
    print(f"Found {len(common_ids)} parallel sentences")

    # Initialize lists
    references = []
    hypotheses_s2tt = []
    hypotheses_t2tt = []
    predicted_s2s = []
    predicted_t2s = []

    for sentence_id in common_ids:
        src = src_by_id[sentence_id]
        tgt = tgt_by_id[sentence_id]

        # Extract source audio & text, and target text
        src_audio = src["audio"]["array"]
        src_sr = src["audio"]["sampling_rate"]
        src_text = src["transcription"]
        tgt_text = tgt["transcription"]

        # Reference list expects list of references per example
        references.append([tgt_text])

        # Resample if needed
        if src_sr != 16000:
            src_audio = resample_to_16k(src_audio, src_sr)

        # Save audio using sentence_id in filename
        audio_path = f"/tmp/input_{sentence_id}.wav"
        torchaudio.save(audio_path, torch.tensor(src_audio).unsqueeze(0), 16000)

        # Speech-to-text translation (S2TT)
        s2tt_out, _ = translator.predict(
            input=audio_path,
            task_str="s2tt",
            tgt_lang=sm4t_tgt_lang
        )
        hypotheses_s2tt.append(s2tt_out[0])

        # Text-to-text translation (T2TT)
        t2tt_out, _ = translator.predict(
            input=src_text,
            task_str="t2tt",
            src_lang=sm4t_src_lang,
            tgt_lang=sm4t_tgt_lang
        )
        hypotheses_t2tt.append(t2tt_out[0])

        # Speech-to-speech translation (S2ST) + ASR on S2ST output
        _, s2s_audio_out = translator.predict(
            input=audio_path,
            task_str="s2st",
            tgt_lang=sm4t_tgt_lang
        )
        s2s_path = f"/tmp/s2s_{sentence_id}.wav"
        torchaudio.save(
            s2s_path,
            s2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
            s2s_audio_out.sample_rate
        )
        s2s_asr_out, _ = translator.predict(
            input=s2s_path,
            task_str="asr",
            tgt_lang=sm4t_tgt_lang
        )
        predicted_s2s.append(s2s_asr_out[0])

        # Text-to-speech translation (T2ST) + ASR on T2ST output
        _, t2s_audio_out = translator.predict(
            input=src_text,
            task_str="t2st",
            src_lang=sm4t_src_lang,
            tgt_lang=sm4t_tgt_lang
        )
        t2s_path = f"/tmp/t2s_{sentence_id}.wav"
        torchaudio.save(
            t2s_path,
            t2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
            t2s_audio_out.sample_rate
        )
        t2s_asr_out, _ = translator.predict(
            input=t2s_path,
            task_str="asr",
            tgt_lang=sm4t_tgt_lang
        )
        predicted_t2s.append(t2s_asr_out[0])

    # Store results
    references_all[sm4t_tgt_lang] = references
    hypotheses_s2tt_all[sm4t_tgt_lang] = hypotheses_s2tt
    hypotheses_t2tt_all[sm4t_tgt_lang] = hypotheses_t2tt
    predicted_text_from_s2s_all[sm4t_tgt_lang] = predicted_s2s
    predicted_text_from_t2s_all[sm4t_tgt_lang] = predicted_t2s

    print(f"✅ [{sm4t_tgt_lang}] Processed {len(common_ids)} parallel sentences")



🔹 Processing Target Language: TEL (te_in)


/scratch/aj/Bhavna/bhav_venv_311/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Found 228 parallel sentences
✅ [tel] Processed 228 parallel sentences

🔹 Processing Target Language: URD (ur_pk)
Found 176 parallel sentences
✅ [urd] Processed 176 parallel sentences


In [21]:
print(references_all)

{'tel': [['రొమాంటిక్ వాదంలో సాంస్కృతిక నిర్ణయాల్లో పెద్ద అంశం కలదు గోతే ఫిచ్టే ష్లేగెల్ వంటి రచయితల నుండి తీసుకోబడింది'], ['కోతల కోసం ఆయన ఒక సంఖ్యను అనుకోలేదు అవి చైనా ఆర్ధిక ఉత్పత్తి ఆధారంగా తయారు చేయబడతాయి'], ['డాక్యుమెంట్ లీక్ ప్రకారం సరిహద్దుల వివాదాన్ని సూచిస్తుంది ఇది 1967 మిడ్ఈస్ట్ యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా పాలస్తీనా కోరుకుంది'], ['మీరు మీ స్వంతం ఆలోచనలతో కాకుండా ప్రభుత్వాల సలహా కూడా తీసుకోవాలని అనుకోవచ్చు అయితే వారి సలహా వారి పౌరుల కోసం రూపొందించబడింది'], ['దక్షిణ ఆఫ్రికాలోని కొన్ని పార్కులకు లేదా అన్ని దక్షిణ ఆఫ్రికా నేషనల్ పార్కులకు ప్రవేశం కలిగించే ఒక వైల్డ్ కార్డును కొనుగోలు చేయడం లాభదాయకం'], ['నిందితుడు ముఖకవతలతో కోర్టులో హాజరయినట్లు పోలీసు ఉన్నతాధికారి చంద్ర శేఖర్ సోలంకి తెలిపారు'], ['"వాటి ఉష్ణ ప్రవర్తన భూమిపై తరచూ స్థిరమైన ఉష్ణోగ్రతను కలిగి ఉండే పెద్ద గుహల వలె స్థిరంగా ఉండదు అవి  నిర్వహిస్తాయి అయితే ఇవి భూమిలో లోతైన రంధ్రాలుగా ఉండటానికి అనుగుణంగా ఉంటాయి "అని అరిజోనాలోని ఫ్లాగ్\u200cస్టాఫ్\u200cలో ఉన్న ఉత్తర అరిజోనా విశ్వవిద్యాలయానికి చెందిన యునైటెడ్ స్టేట్స్ జి

In [23]:
print(hypotheses_s2tt_all)

{'tel': [CString('రొమానీతంలో సాంస్కృతిక నైతికవాదం యొక్క పెద్ద మూలకం ఉంది ఇది గోథే ఫిచ్టే మరియు స్లేగల్ వంటి రచయితల నుండి తీసుకోబడింది'), CString('ఇది చెప్తూ వారు చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధారంగా తయారు చేయబడతారు ఆయన కోత కోసం ఏ గణాంకం నిర్ణయించలేదు'), CString('లీగ్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని ప్రస్తావిస్తుంది ఇది పాలస్తీనా పంతొమ్మిది వందల అరవైల మధ్యప్రాచ్య యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా కోరుకుంటుంది'), CString('మీ ప్రభుత్వం కాకుండా మీరు ఇతర దేశాల ప్రభుత్వాల సలహాలు తీసుకోవచ్చు అయితే వారి సలహాలు వారి పౌరులను దృష్టిలో ఉంచుకొని ఇవ్వబడుతుంది'), CString('వైల్డ్ కార్డ్ కొనుగోలు తీసుకోవడం లాభదాయకం కూడా కావచ్చు ఎందుకంటే దీనితో మీరు దక్షిణాఫ్రికా యొక్క చెరిండా పార్క్ లేదా అన్ని దక్షిణాఫ్రికా నేషనల్ పార్క్ లు చూడవచ్చు'), CString('పోలీస్ అధిపతి చంద్రశేఖర్ సోలంకి చెప్పారు, ఆరోపణలు కవర్ చేసిన ముఖంతో కోర్టులో కనిపించారు.'), CString('ఆయన ఉష్మీయ ప్రవర్తన భూమి యొక్క పెద్ద గుహలు లాంటిది కాదు ఇవి తరచుగా చాలా స్థితిస్థాయి ఉష్ణోగ్రతలను కలిగి ఉంటాయి కానీ ఇది భూమి యొక్క ఈ లోతైన రంధ్రాలకు అనుగు

In [24]:
print(hypotheses_t2tt_all)

{'tel': [CString('రొమానియం లో సాంస్కృతిక నిర్ణీతవాదం యొక్క పెద్ద అంశం ఉంది అది గోథే ఫిచ్టే మరియు ష్లెగల్ వంటి రచయితల నుండి తీసుకోబడింది'), CString('అవి చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధారంగా తయారు చేయబడతాయని చెప్పడం వారు తగ్గింపు కోసం ఏ గణాంకాన్ని నిర్ణయించలేదు'), CString('లీక్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని సూచిస్తుంది ఇది పాలస్తీనా 1967 యొక్క మధ్యప్రాచ్య యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా కోరుకుంటుంది'), CString('మీ ప్రభుత్వానికి అదనంగా మీరు ఇతర దేశాల ప్రభుత్వాల సలహాలను తీసుకోవచ్చు అయితే వారి సలహా వారి పౌరులను దృష్టిలో ఉంచుకుని ఇవ్వబడతాయి'), CString('వైల్డ్ కార్డ్ కొనుగోలు చేసుకోవడం ప్రయోజనకరమైనది కూడా కావచ్చు ఎందుకంటే దీని ద్వారా మీరు దక్షిణాఫ్రికా యొక్క ఎంపికైన పార్క్ లేదా అన్ని దక్షిణాఫ్రికా నేషనల్ పార్క్ ను చూడవచ్చు'), CString('పోలీస్ సూపరింటెండెంట్ చంద్ర షేఖర్ సోలంకి చెప్పారు ఆరోపణలు కప్పబడిన ముఖంతో కోర్టులో హాజరయ్యారు'), CString('వారి ఉష్ణ ప్రవర్తన భూమి యొక్క పెద్ద గుహల మాదిరిగా స్థిరంగా లేదు ఇది తరచుగా చాలా స్థిరమైన ఉష్ణోగ్రతను నిర్వహిస్తుంది కానీ ఇది భూమి యొక్క ఈ లోతైన ర

In [28]:
print(hypotheses_s2tt_all)

{'tel': [CString('రొమానీతంలో సాంస్కృతిక నైతికవాదం యొక్క పెద్ద మూలకం ఉంది ఇది గోథే ఫిచ్టే మరియు స్లేగల్ వంటి రచయితల నుండి తీసుకోబడింది'), CString('ఇది చెప్తూ వారు చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధారంగా తయారు చేయబడతారు ఆయన కోత కోసం ఏ గణాంకం నిర్ణయించలేదు'), CString('లీగ్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని ప్రస్తావిస్తుంది ఇది పాలస్తీనా పంతొమ్మిది వందల అరవైల మధ్యప్రాచ్య యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా కోరుకుంటుంది'), CString('మీ ప్రభుత్వం కాకుండా మీరు ఇతర దేశాల ప్రభుత్వాల సలహాలు తీసుకోవచ్చు అయితే వారి సలహాలు వారి పౌరులను దృష్టిలో ఉంచుకొని ఇవ్వబడుతుంది'), CString('వైల్డ్ కార్డ్ కొనుగోలు తీసుకోవడం లాభదాయకం కూడా కావచ్చు ఎందుకంటే దీనితో మీరు దక్షిణాఫ్రికా యొక్క చెరిండా పార్క్ లేదా అన్ని దక్షిణాఫ్రికా నేషనల్ పార్క్ లు చూడవచ్చు'), CString('పోలీస్ అధిపతి చంద్రశేఖర్ సోలంకి చెప్పారు, ఆరోపణలు కవర్ చేసిన ముఖంతో కోర్టులో కనిపించారు.'), CString('ఆయన ఉష్మీయ ప్రవర్తన భూమి యొక్క పెద్ద గుహలు లాంటిది కాదు ఇవి తరచుగా చాలా స్థితిస్థాయి ఉష్ణోగ్రతలను కలిగి ఉంటాయి కానీ ఇది భూమి యొక్క ఈ లోతైన రంధ్రాలకు అనుగు

In [25]:
print(predicted_text_from_s2s_all)

{'tel': [CString('రొమానీతంలో సాంస్కృతిక నైతికవాదం యొక్క పెద్ద మూలకం ఉంది ఇది గోదే ఫిట్టే మరియు స్లేగల్ వంటి రచయితల నుండి తీసుకోబడింది.'), CString('ఇది చెప్తూ వారు చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధారంగా తయారు చేయబడతారు. ఆయన కోత కోసం ఏ గణాంకం నిర్ణయించలేదు.'), CString('లీగ్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని ప్రస్తావిస్తుంది. ఇది పాలస్తీనా పంతొమ్మిది వందల అరవైల మధ్యప్రాచ్య యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా కోరుకుంటుంది.'), CString('మీ ప్రభుత్వం కాకుండా మీరు ఇతర దేశాల ప్రభుత్వాల సలహాలు తీసుకోవచ్చు అయితే వారి సలహాలు వారి పౌరులను దృష్టిలో ఉంచుకొని ఇవ్వబడుతుంది'), CString('వైల్డ్ కార్డ్ కొనుగోలు తీసుకోవడం లాభదాయకం కూడా కావచ్చు ఎందుకంటే దీనితో మీరు దక్షిణాఫ్రికా యొక్క చెరిండా పార్క్ లేదా అన్ని దక్షిణాఫ్రికా నేషనల్ పార్కులను చూడవచ్చు.'), CString('పోలీస్ అధిపతి చంద్రశేఖర్ సోలంకి చెప్పారు ఆరోపణలు కవర్ చేసిన ముఖంతో కోర్టులో కనిపించారు'), CString('ఆయన ఉష్మీయ ప్రవర్తన భూమి యొక్క పెద్ద గుహలు లాంటిది కాదు ఇవి తరచూ చాలా స్థితిస్థాపక ఉష్ణోగ్రతలను కలిగి ఉంటాయి కానీ ఇది భూమి యొక్క ఈ లోతైన రంధ్రాలకు అను

In [26]:
print(predicted_text_from_t2s_all)

{'tel': [CString('రొమేనియంలో సాంస్కృతిక నిర్మితవాదం యొక్క పెద్ద అంశం ఉంది అది గోథే ఫిట్టే మరియు ష్లేగల్ వంటి రచయితల నుండి తీసుకోబడింది.'), CString('అవి చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధారంగా తయారవుతాయని చెప్పడం వారు తగ్గింపు కోసం ఏ గణాంకాన్ని నిర్ణయించలేదు.'), CString('లీగ్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని సూచిస్తుంది ఇది పాలస్తీనా నైతిక నిర్మూలన యొక్క మధ్యప్రాచ్య యుద్ధానికి ముందు సరిహద్దుల ఆధారంగా కోరుకుంటుంది'), CString('మీ ప్రభుత్వానికి అదనంగా మీరు ఇతర దేశాల ప్రభుత్వాల సలహాలను తీసుకోవచ్చు, అయితే వారి సలహా వారి పౌరులను దృష్టిలో ఉంచుకొని ఇవ్వబడతాయి.'), CString('వైల్డ్ కార్డ్ కొనుగోలు చేసుకోవడం ప్రయోజనకరమైనది కూడా కావచ్చు ఎందుకంటే దీని ద్వారా మీరు దక్షిణాఫ్రికా యొక్క ఎన్నుకోబడిన పార్క్ లేదా అన్ని దక్షిణాఫ్రికా నేషనల్ పార్క్ ను చూడవచ్చు.'), CString('పోలీస్ సూపరింటెండెంట్ చంద్రశేఖర్ సోలంకి చెప్పారు ఆరోపణలు కప్పబడిన ముఖంతో కోర్టులో హాజరయ్యారు'), CString('వారి ఉష్ణ ప్రవర్తన భూమి యొక్క పెద్ద గుహల మాదిరిగా స్థిరంగా లేదు. ఇది చాలా తరచుగా చాలా స్థిరమైన ఉష్ణోగ్రతలను నిర్వహిస్తుంది కానీ ఇది భ

In [ ]:
from datasets import load_dataset
import torch
import torchaudio
import pandas as pd

# Dictionaries to store all outputs
references_all = {}
hypotheses_s2tt_all = {}
hypotheses_t2tt_all = {}
predicted_text_from_s2s_all = {}
predicted_text_from_t2s_all = {}

# Language mapping: SeamlessM4T tgt_lang → (FLEURS target_lang)
lang_map = {
    "tel": "te_in",   # Telugu
    "urd": "ur_pk"    # Urdu (Pakistan)
}

# Fixed source language
fleurs_src_lang = "hi_in"  # Hindi in FLEURS
sm4t_src_lang = "hin"      # Hindi in SeamlessM4T

# Resampling function to 16 kHz
def resample_to_16k(audio, orig_sr):
    return torchaudio.transforms.Resample(orig_sr, 16000)(torch.tensor(audio)).numpy()

# Loop through each target language
for sm4t_tgt_lang, fleurs_tgt_lang in lang_map.items():
    print("\n" + "="*50)
    print(f"🔹 Processing Target Language: {sm4t_tgt_lang.upper()} ({fleurs_tgt_lang})")
    print("="*50)

    # Load FLEURS test splits
    src_dataset = load_dataset("google/fleurs", fleurs_src_lang, split="test")
    tgt_dataset = load_dataset("google/fleurs", fleurs_tgt_lang, split="test")

    # Build mapping from sentence ID to record
    src_by_id = {item["id"]: item for item in src_dataset}
    tgt_by_id = {item["id"]: item for item in tgt_dataset}
    common_ids = sorted(set(src_by_id.keys()) & set(tgt_by_id.keys()))
    print(f"Found {len(common_ids)} parallel sentences")

    # Initialize lists
    references = []
    hypotheses_s2tt = []
    hypotheses_t2tt = []
    predicted_s2s = []
    predicted_t2s = []
    source_texts = []

    for sentence_id in common_ids:
        src = src_by_id[sentence_id]
        tgt = tgt_by_id[sentence_id]

        src_audio = src["audio"]["array"]
        src_sr = src["audio"]["sampling_rate"]
        src_text = src["transcription"]
        tgt_text = tgt["transcription"]

        references.append([tgt_text])
        source_texts.append(src_text)

        # Resample if needed
        if src_sr != 16000:
            src_audio = resample_to_16k(src_audio, src_sr)

        # Save source audio temporarily
        audio_path = f"/tmp/input_{sentence_id}.wav"
        torchaudio.save(audio_path, torch.tensor(src_audio).unsqueeze(0), 16000)

        # Speech-to-text translation (S2TT)
        s2tt_out, _ = translator.predict(
            input=audio_path,
            task_str="s2tt",
            tgt_lang=sm4t_tgt_lang
        )
        hypotheses_s2tt.append(s2tt_out[0])

        # Text-to-text translation (T2TT)
        t2tt_out, _ = translator.predict(
            input=src_text,
            task_str="t2tt",
            src_lang=sm4t_src_lang,
            tgt_lang=sm4t_tgt_lang
        )
        hypotheses_t2tt.append(t2tt_out[0])

        # Speech-to-speech translation (S2ST) + ASR
        _, s2s_audio_out = translator.predict(
            input=audio_path,
            task_str="s2st",
            tgt_lang=sm4t_tgt_lang
        )
        s2s_path = f"/tmp/s2s_{sentence_id}.wav"
        torchaudio.save(
            s2s_path,
            s2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
            s2s_audio_out.sample_rate
        )
        s2s_asr_out, _ = translator.predict(
            input=s2s_path,
            task_str="asr",
            tgt_lang=sm4t_tgt_lang
        )
        predicted_s2s.append(s2s_asr_out[0])

        # Text-to-speech translation (T2ST) + ASR
        _, t2s_audio_out = translator.predict(
            input=src_text,
            task_str="t2st",
            src_lang=sm4t_src_lang,
            tgt_lang=sm4t_tgt_lang
        )
        t2s_path = f"/tmp/t2s_{sentence_id}.wav"
        torchaudio.save(
            t2s_path,
            t2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
            t2s_audio_out.sample_rate
        )
        t2s_asr_out, _ = translator.predict(
            input=t2s_path,
            task_str="asr",
            tgt_lang=sm4t_tgt_lang
        )
        predicted_t2s.append(t2s_asr_out[0])

    # Store results per language
    references_all[sm4t_tgt_lang] = references
    hypotheses_s2tt_all[sm4t_tgt_lang] = hypotheses_s2tt
    hypotheses_t2tt_all[sm4t_tgt_lang] = hypotheses_t2tt
    predicted_text_from_s2s_all[sm4t_tgt_lang] = predicted_s2s
    predicted_text_from_t2s_all[sm4t_tgt_lang] = predicted_t2s

    # Build final DataFrame
    df = pd.DataFrame({
        "source_text": source_texts,
        "reference_text": [r[0] for r in references],
        "S2TT_prediction": hypotheses_s2tt,
        "T2TT_prediction": hypotheses_t2tt,
        "S2ST_ASR": predicted_s2s,
        "T2ST_ASR": predicted_t2s
    })

    print(f"✅ [{sm4t_tgt_lang}] Processed {len(common_ids)} parallel sentences")
    print(df.head(5))



🔹 Processing Target Language: TEL (te_in)


/scratch/aj/Bhavna/bhav_venv_311/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Found 228 parallel sentences
✅ [tel] Processed 228 parallel sentences
                                         source_text  \
0  रूमानियत में सांस्कृतिक नियतिवाद का बड़ा तत्व ...   
1  यह कहते हुए कि वे चीन के आर्थिक उत्पादन के आधा...   
2  लीक के अनुसार दस्तावेज़ सीमा विवाद को संदर्भित...   
3  अपनी सरकार के अलावा आप अन्य देशों की सरकारों क...   
4  वाइल्ड कार्ड खरीद लेना फ़ायदेमंद भी हो सकता है...   

                                      reference_text  \
0  రొమాంటిక్ వాదంలో సాంస్కృతిక నిర్ణయాల్లో పెద్ద ...   
1  కోతల కోసం ఆయన ఒక సంఖ్యను అనుకోలేదు అవి చైనా ఆర...   
2  డాక్యుమెంట్ లీక్ ప్రకారం సరిహద్దుల వివాదాన్ని ...   
3  మీరు మీ స్వంతం ఆలోచనలతో కాకుండా ప్రభుత్వాల సలహ...   
4  దక్షిణ ఆఫ్రికాలోని కొన్ని పార్కులకు లేదా అన్ని...   

                                     S2TT_prediction  \
0  రొమానీతంలో సాంస్కృతిక నైతికవాదం యొక్క పెద్ద మూ...   
1  ఇది చెప్తూ వారు చైనా యొక్క ఆర్థిక ఉత్పత్తి ఆధా...   
2  లీగ్ ప్రకారం డాక్యుమెంట్ సరిహద్దు వివాదాన్ని ప...   
3  మీ ప్రభుత్వం కాకుండా మీరు ఇతర

/scratch/aj/Bhavna/bhav_venv_311/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Found 176 parallel sentences
✅ [urd] Processed 176 parallel sentences
                                         source_text  \
0  रूमानियत में सांस्कृतिक नियतिवाद का बड़ा तत्व ...   
1  अपनी सरकार के अलावा आप अन्य देशों की सरकारों क...   
2  आम शब्दों में जब प्रबंधक अपने पूर्व सहयोगियों ...   
3  वाइल्ड कार्ड खरीद लेना फ़ायदेमंद भी हो सकता है...   
4  सावधान रहें कि कपड़े को बहुत गर्म न होने दें ज...   

                                      reference_text  \
0  رومانویت میں ثقافتی عزم کا ایک بہت بڑا عنصر ہو...   
1  آپ اپنی حکومت کے علاوہ دوسری حکومتوں کا مشورہ ...   
2  عام بات کریں، تو اپنے سابق ساتھیوں کی قیادت کر...   
3  یہ کسی کیلئے ایک وائلڈ کارڈ خریدنے کے لحاظ سے ...   
4  احتیاط رکھیں کہ آپ کا کپڑا زیادہ گرم نہ ہونے پ...   

                                     S2TT_prediction  \
0  رومانیت میں ثقافتی اخلاقیات کا بڑا عنصر تھا جو...   
1  اپنی حکومت کے علاوہ آپ دیگر ممالک کی حکومتوں ک...   
2  اہم الفاظ میں جب منتظم اپنے سابق ساتھیوں کی قی...   
3  وائلڈ کارڈ خرید لینا فائدہ من

: 

In [ ]:
import pandas as pd

# Pick a target language, e.g., Telugu
lang_code = "tel"
# Build mapping from sentence ID to record
src_by_id = {item["id"]: item for item in src_dataset}
tgt_by_id = {item["id"]: item for item in tgt_dataset}
common_ids = sorted(set(src_by_id.keys()) & set(tgt_by_id.keys()))

# Extract lists
source_texts = []
reference_texts = []
s2tt_hyps = hypotheses_s2tt_all[lang_code]  # speech-to-text predictions
t2tt_hyps = hypotheses_t2tt_all[lang_code]  # text-to-text predictions
s2st_asr = predicted_text_from_s2s_all[lang_code]  # speech-to-speech ASR
t2st_asr = predicted_text_from_t2s_all[lang_code]  # text-to-speech ASR

for sentence_id in common_ids:
    src = src_by_id[sentence_id]
    tgt = tgt_by_id[sentence_id]

    source_texts.append(src["transcription"])
    reference_texts.append(tgt["transcription"])

# Create DataFrame
df = pd.DataFrame({
    "source_text": source_texts,
    "reference_text": reference_texts,
    "S2TT_prediction": s2tt_hyps,
    "T2TT_prediction": t2tt_hyps,
    "S2ST_ASR": s2st_asr,
    "T2ST_ASR": t2st_asr
})

# Show first 5 rows
print(df.head(5))


In [20]:
from sacrebleu import corpus_bleu, corpus_chrf
from jiwer import wer
from pprint import pprint

# Container for all language metrics
all_metrics = {}

for lang_code in references_all.keys():
    references = references_all[lang_code]
    hypotheses_s2tt = hypotheses_s2tt_all[lang_code]
    hypotheses_t2tt = hypotheses_t2tt_all[lang_code]
    predicted_text_from_s2s = predicted_text_from_s2s_all[lang_code]
    predicted_text_from_t2s = predicted_text_from_t2s_all[lang_code]

    # --- Normalize hypotheses ---
    hypotheses_s2tt = [str(h).strip() for h in hypotheses_s2tt]
    hypotheses_t2tt = [str(h).strip() for h in hypotheses_t2tt]
    predicted_text_from_s2s = [str(h).strip() for h in predicted_text_from_s2s]
    predicted_text_from_t2s = [str(h).strip() for h in predicted_text_from_t2s]

    # --- Normalize references (SacreBLEU expects list-of-lists) ---
    references_norm = [[str(r[0]).strip()] for r in references]
    multi_references = list(zip(*references_norm))  # shape: (num_refs, num_sentences)

    metrics = {}

    # --- BLEU / chrF2++ ---
    metrics["S2TT_SacreBLEU"] = corpus_bleu(hypotheses_s2tt, multi_references).score
    metrics["T2TT_chrF2++"] = corpus_chrf(hypotheses_t2tt, multi_references).score
    metrics["S2ST_ASR_BLEU"] = corpus_bleu(predicted_text_from_s2s, multi_references).score
    metrics["T2ST_ASR_BLEU"] = corpus_bleu(predicted_text_from_t2s, multi_references).score

    # --- WER (jiwer) ---
    metrics["S2ST_ASR_WER"] = sum(
        wer(ref[0], hyp) for ref, hyp in zip(references_norm, predicted_text_from_s2s)
    ) / len(references_norm)

    metrics["T2ST_ASR_WER"] = sum(
        wer(ref[0], hyp) for ref, hyp in zip(references_norm, predicted_text_from_t2s)
    ) / len(references_norm)

    metrics["S2TT_WER"] = sum(
        wer(ref[0], hyp) for ref, hyp in zip(references_norm, hypotheses_s2tt)
    ) / len(references_norm)

    metrics["T2TT_WER"] = sum(
        wer(ref[0], hyp) for ref, hyp in zip(references_norm, hypotheses_t2tt)
    ) / len(references_norm)

    # Print + save
    print(f"\n=== Metrics for target language: {lang_code.upper()} ===")
    pprint(metrics)

    all_metrics[lang_code] = metrics



=== Metrics for target language: TEL ===
{'S2ST_ASR_BLEU': 6.703186032040358,
 'S2ST_ASR_WER': 0.9207014212978243,
 'S2TT_SacreBLEU': 6.791687079616478,
 'S2TT_WER': 0.9214235330398954,
 'T2ST_ASR_BLEU': 9.072877840222192,
 'T2ST_ASR_WER': 0.8816193817330296,
 'T2TT_WER': 0.85229959325962,
 'T2TT_chrF2++': 49.22213115874377}

=== Metrics for target language: URD ===
{'S2ST_ASR_BLEU': 14.802831740006779,
 'S2ST_ASR_WER': 0.7457257921461948,
 'S2TT_SacreBLEU': 14.627013884604041,
 'S2TT_WER': 0.7461084350137203,
 'T2ST_ASR_BLEU': 16.53967566437534,
 'T2ST_ASR_WER': 0.7231635694724213,
 'T2TT_WER': 0.7154703562371476,
 'T2TT_chrF2++': 44.07527441374375}


In [1]:
import torch
print(f"\nTorch version: {torch.__version__}")


Torch version: 2.5.1+cu121


In [2]:
%pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 32.0 MB/s  0:00:24:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.4 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 48.6 MB/s  0:00:13:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 55.7 MB/s  0:00:02:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 54.5 MB/s  0:00:03:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: nvidia-nccl-cu12━━━━━━━━ 0/6 [triton]
    Found existing installation: nvidia-nccl-cu12 2.21.56 [triton]
    Uninstalling nvidia-nccl-cu12-2.21.5:━━━ 0/6 [triton]
      Successfully uninstalled nvidia-nccl-cu12-2.21.50/6 [triton]
  Attem